<h2>If the notebook helps you in learning new things and getting introduced to any new concepts,please do upvote the notebook.

- Any improvement suggestions would be very much helpful

<h2>Tabular Playground Series-April'21</h2>

In the notebook, a series of steps in being followed in order to attain desired accuracy.
The steps being taken are as follows:
-  Normal Data Exploration
- Exploratory Data Analysis
- NULL Value Treatment
- Feature Transformation
- Splitting and Testing models 
- Generating the results

<h1><b>1. Normal Data Exploration</b></h1>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv')


train.head()

In [ ]:
train.info()

<h3>As from the above information displayed, it can be clearly seen that there are NaN values in the dataset and hence the data needs further NULL value treatment and this is done after performing EDA</h3>

<h1><b>2. Exploratory Data Analysis</b></h1>

In [ ]:
##Null values are present hence we need to do NULL Value treatment at an initial stage.
#num = [x for x in train.columns if train[x].dtypes!='O']
for i in ['Age','Fare']:
    sns.boxplot(train[i])
    plt.show()

In [ ]:
##NULL Values Heatmap Visualization
sns.heatmap(train.isnull())

In [ ]:
sns.distplot(train.Age,hist = False)
##It can be clearly seen from the distplot that most of the people were belonging to the age group of >20 and <80

In [ ]:
# Here no class imbalance is found in case of the target variable and the disrtibution can be seen as follows
sns.countplot(train['Survived'])

In [ ]:
##No class imbalance as such in case of Gender
sns.countplot(train['Sex'],hue = train['Survived'])

In [ ]:
##This can be clearly observerd from the plot that most of the values for Embarked are S followed by C and then Q which are less in number
sns.countplot(train['Embarked'],hue = train['Survived'])

In [ ]:
##For most of the families, there were families with 0 siblings together and very few of them had 1 or 2 siblings.
# Count is found to be very less for more than 2 siblings
sns.countplot(train['SibSp'],hue = train['Survived'])

In [ ]:
##This value of Parch was also found to be 0 for most of the times and rest of the values are very less in number
sns.countplot(train['Parch'])

<h2><b>3. NULL Value Treatment</b></h2>

- <h3>Keeping the above plots in mind and the distribution of all variables,  appropriate values are being used for replacing with the NULL values</h3>

In [ ]:
##Removing NULL values from the Age 
test['Age'].fillna((train['Age'].median()), inplace=True)
train['Age'].fillna((train['Age'].median()), inplace=True)

##bfill is being done instead of any particular value since using any constant value at times reduced accuracy
# test['Embarked'].fillna(method = 'bfill', inplace=True)
# train['Embarked'].fillna(method = 'bfill', inplace=True)

test['Embarked'].fillna((train['Embarked'].mode()), inplace=True)
train['Embarked'].fillna((train['Embarked'].mode()), inplace=True)

##Removal of NULL values from Fare Attribute
test['Fare'].fillna((train['Fare'].mean()), inplace=True)
train['Fare'].fillna((train['Fare'].mean()), inplace=True)

<h3><b>Creating Extra Features</b></h3>

- At times creating extra features using existing features in the dataset increases the overall model accuracy.
- The new features being generated are based on the there correlation values.

In [ ]:
##Creating new, extra features from increasing accuracy. 
train['Age*Fare'] = train['Age']*train['Fare']
test['Age*Fare'] = test['Age']*test['Fare']


train['Age/Fare'] = train['Age']/train['Fare']
test['Age/Fare'] = test['Age']/test['Fare']


train['FamilySize'] = train['SibSp'] + train['Parch'] + 1 
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1 

##Dropping unnecessary Categorical features. 
train.drop(['Name','Cabin','PassengerId','Ticket'],axis = 1,inplace = True)
test.drop(['Name','Cabin','PassengerId','Ticket'],axis = 1,inplace = True)


Also a common observation is being seen that:
- For most of the instances, both the columns, Ticket and Cabin were found to be NULL at the same time. and hence can be considered for dropping.

<h3> Now that we have done all the required things for NULL value treatment, we further go on to work with Encoding Techniques in order to encode the available categorical features to convert them to numerical ones</h3>

In [ ]:
cate = [x for x in train.columns if train[x].dtypes == 'O']
cate1 = [x for x in test.columns if test[x].dtypes == 'O']

for i in cate:
    label = LabelEncoder()
    train[i] = label.fit_transform(train[i])
    test[i] = label.transform(test[i])
    

X = train.drop('Survived',axis = 1)
y = train.Survived

In [ ]:
##Correlation Heatmap
plt.figure(figsize=(8, 8))
sns.heatmap(train.corr(),annot = True)

<h2><b>4. Feature Transformation</b></h2>

Here various trials of different types of scaling techniques were being tried. Some amongst them are as:
- 1. MinMaxScaler
- 2. QuantileTransformer
- 3. PowerTransformer
- 4. RobustScaler.

All of the above scaling techniques should be applied to the data and this is much useful in increasing accuracy for Linear models. 

In [ ]:
from sklearn.preprocessing import RobustScaler,PowerTransformer
scaler = PowerTransformer()
X2 = scaler.fit_transform(X)
test3 = scaler.transform(test)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X2,y)

<h2>5. Basic Modelling</h2>

In [ ]:
from sklearn.linear_model import LogisticRegression 
mdl = LogisticRegression().fit(X2,y)
mdl.score(X_test,y_test)
#0.76932

In [ ]:
pred = mdl.predict(test3)

In [ ]:
pred = pd.DataFrame(pred,columns = ['Survived'])
test = pd.concat([pred,test],axis = 1)

clms = [train,test]
final = pd.concat(clms)

- <h2>Extra Attempt: PsuedoLabelling</h2>

- Using Results predicted on test data from models for actual predictions

In [ ]:
xf = final.drop('Survived',axis = 1)
yf = final.Survived

test11 = test.drop('Survived',axis = 1)

In [ ]:
from sklearn.preprocessing import RobustScaler,PowerTransformer
scalerf = PowerTransformer()
X2f = scalerf.fit_transform(xf)
test3 = scalerf.transform(test11)

from sklearn.model_selection import train_test_split
X_trainf,X_testf,y_trainf,y_testf = train_test_split(X2f,yf)

In [ ]:
from sklearn.linear_model import LogisticRegression 

mdl_final = LogisticRegression().fit(X2f,yf)
mdl_final.score(X_testf,y_testf)
#0.76932

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(max_depth = 6)
xgb.fit(X2f,yf)
xgb.score(X_testf,y_testf)

<h2>StackingClassifier</h2>

- An efficient ensemble technique that can used to combine different models stacking them one on other 
- Various models could be combined together and a final model to make predictions on the one made by all other models results in the overall prediction and hence increases the overall performance

In [ ]:
from sklearn.ensemble import StackingClassifier

estimators = [('m1',mdl_final),('ridge',xgb)]
clf_mdl = StackingClassifier(estimators = estimators,final_estimator = XGBClassifier() )
clf_mdl.fit(X2f,yf)

<h2>6. Generating Results for Submission</h2>

In [ ]:
##Generating the results for submission
predxgb = xgb.predict(test3)
predxgb = pd.DataFrame(predxgb,columns = ['Survived'])

testt = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv')
res = pd.DataFrame(predxgb,columns = ['Survived'])
res = pd.concat([testt.PassengerId,res],axis = 1)
#res.to_csv('14_2.csv',index = False)

In [ ]:
##Generating the results for submission
predxgb = clf_mdl.predict(test3)
predxgb = pd.DataFrame(predxgb,columns = ['Survived'])

testt = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv')
res = pd.DataFrame(predxgb,columns = ['Survived'])
res = pd.concat([testt.PassengerId,res],axis = 1)
#res.to_csv('16_1.csv',index = False)

In [ ]:
res.head()

In [ ]:
#Using LGBM
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

lgbm = LGBMClassifier(boosting_type = 'dart',num_leaves = 32,max_depth = 8,colsample_bytree = 0.8,extra_trees = True,n_jobs = -1)
lgbm.fit(X2f, yf)
y_pred = lgbm.predict(test3).astype(int)




In [ ]:
##Combining all best models
est_vote = [('m1',clf_mdl),('m2',mdl_final),('ridge',xgb),('m3',lgbm)]
clf_vote = VotingClassifier(estimators = est_vote,voting = 'soft')
clf_vote.fit(X2f,yf)

In [ ]:
predictions = clf_vote.predict(test3)


In [ ]:
##Generating the results for submission
predvote = clf_vote.predict(test3)
predvote = pd.DataFrame(predvote,columns = ['Survived'])

testt = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv')
res1 = pd.DataFrame(predvote,columns = ['Survived'])
res1 = pd.concat([testt.PassengerId,res1],axis = 1)
res1.to_csv('17.csv',index = False)

<h2> More updates will be made soon!</h2>